In [1]:
from mypulp import *
import numpy as np
import itertools
import networkx as nx
import math
import matplotlib.pyplot as plt
%matplotlib inline

## 制約付き最短路問題 (CSP)

In [57]:
# CSPを解いた結果を返す関数
def solve_CSP(V, E, u):
    model = Model('csp')
    
    ## 変数
    x = {}
    for i, j in E:
        x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
    model.update()
    
    ## 定式化
    ### 制約式
    model.addConstr(quicksum(c[i, j] * x[i, j] for i, j in E) <= u)
    
    for i in V[1: -1]:
        delta_p = [(i_, j) for i_, j in E if i_==i]  # ノードiから出る枝の集合
        delta_m = [(j, i_) for j, i_ in E if i_==i]  # ノードiに入る枝の集合
        model.addConstr(quicksum(x[i_, j] for i_, j in delta_p)
                        - quicksum(x[i_, j] for i_, j in delta_m) == 0)
                        
    model.addConstr(quicksum(x[i, j] for i, j in E if i==V[0]) == 1)
    model.addConstr(quicksum(x[i, j] for i, j in E if j==V[-1]) == 1)
 
    ### 目的関数
    model.setObjective(quicksum(l[i, j] * x[i, j] for i, j in E), GRB.MINIMIZE)
    
    model.update()
    model.__data = x
    
    return model        

In [122]:
# 例題
V = range(4)
E = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3)]
c = {(0, 1): 0, (0, 2): 2, (1, 2): 9, (1, 3): 9, (2, 3): 5}
l = {(0, 1): 2, (0, 2): 6, (1, 2): 1, (1, 3): 5, (2, 3): 3}
u = 10

In [59]:
# 最適化
model = solve_CSP(V, E, u)
model.optimize()
print(f'Optimal Value: {model.ObjVal}')

x = model.__data
eps = 1.0e-6
optsol = []
for i, j in x:
    if x[i, j].X > eps:
        optsol.append(x[i, j])

print(f'Optimal Solution: {optsol}')

Optimal Value: 7.0
Optimal Solution: [x_0,1, x_1,3]


## CSPの不等式制約に対するラグランジュ緩和問題 (LSP)

In [66]:
# CSPを解いた結果を返す関数
def solve_LSP(V, E, u, sigma):
    model = Model('lsp')
    
    ## 変数
    x = {}
    for i, j in E:
        x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
    model.update()
    
    ## 定式化
    ### 制約式
    for i in V[1: -1]:
        delta_p = [(i_, j) for i_, j in E if i_==i]  # ノードiから出る枝の集合
        delta_m = [(j, i_) for j, i_ in E if i_==i]  # ノードiに入る枝の集合
        model.addConstr(quicksum(x[i_, j] for i_, j in delta_p)
                        - quicksum(x[i_, j] for i_, j in delta_m) == 0)
                        
    model.addConstr(quicksum(x[i, j] for i, j in E if i==V[0]) == 1)
    model.addConstr(quicksum(x[i, j] for i, j in E if j==V[-1]) == 1)
 
    ### 目的関数
    model.setObjective(quicksum(l[i, j] * x[i, j] for i, j in E)
                       + sigma*(quicksum(c[i, j] * x[i, j] for i, j in E) - u), GRB.MINIMIZE)
    
    model.update()
    model.__data = x
    
    return model

In [137]:
# 最適化
model = solve_LSP(V, E, u, sigma=0.2)  # 最強ラグランジュ乗数
model.optimize()
print(f'Optimal Value: {model.ObjVal}')

x = model.__data
eps = 1.0e-6
optsol = []
for i, j in x:
    if x[i, j].X > eps:
        optsol.append(x[i, j])

print(f'Optimal Solution: {optsol}')

Optimal Value: 6.8
Optimal Solution: [x_0,1, x_1,2, x_2,3]


最強ラグランジュ乗数を劣勾配法を用いて探索する

In [170]:
sigma = 0
count = 1
eps = 1.0e-2
converge = False

while not converge:
    # LSPを解く
    model = solve_LSP(V, E, u, sigma=sigma)
    model.optimize()
    x = model.__data
    
    # sigmaの係数（=劣勾配）
    sub_grad = value(quicksum(c[i, j] * x[i, j].X for i, j in E) - u)
    
    # ちゃんと収束するようにsigmaの更新幅は徐々に小さくしていく
    # sigmaの更新幅が微小定数epsよりも小さくなれば終了する
    if np.abs(sub_grad / count) < eps:
        converge = True
        count += 1
    else:
        sigma = np.max([0, sigma + sub_grad/count])
        count += 1

    print(f'{count-1}回目の反複：sigma={sigma: .4f}, Objval={model.ObjVal: .4f}')

1回目の反複：sigma= 4.0000, Objval= 6.0000
2回目の反複：sigma= 2.5000, Objval=-3.0000
3回目の反複：sigma= 1.5000, Objval= 1.5000
4回目の反複：sigma= 0.7500, Objval= 4.5000
5回目の反複：sigma= 0.5500, Objval= 6.2500
6回目の反複：sigma= 0.3833, Objval= 6.4500
7回目の反複：sigma= 0.2405, Objval= 6.6167
8回目の反複：sigma= 0.1155, Objval= 6.7595
9回目の反複：sigma= 0.5599, Objval= 6.4619
10回目の反複：sigma= 0.4599, Objval= 6.4401
11回目の反複：sigma= 0.3690, Objval= 6.5401
12回目の反複：sigma= 0.2857, Objval= 6.6310
13回目の反複：sigma= 0.2088, Objval= 6.7143
14回目の反複：sigma= 0.1373, Objval= 6.7912
15回目の反複：sigma= 0.4040, Objval= 6.5493
16回目の反複：sigma= 0.3415, Objval= 6.5960
17回目の反複：sigma= 0.2827, Objval= 6.6585
18回目の反複：sigma= 0.2271, Objval= 6.7173
19回目の反複：sigma= 0.1745, Objval= 6.7729
20回目の反複：sigma= 0.3745, Objval= 6.6979
21回目の反複：sigma= 0.3269, Objval= 6.6255
22回目の反複：sigma= 0.2814, Objval= 6.6731
23回目の反複：sigma= 0.2379, Objval= 6.7186
24回目の反複：sigma= 0.1963, Objval= 6.7621
25回目の反複：sigma= 0.3563, Objval= 6.7851
26回目の反複：sigma= 0.3178, Objval= 6.6437
27回目の反複：sigma= 0.2808

最終的に得られた解

In [172]:
# 最適化
model = solve_LSP(V, E, u, sigma=sigma)
model.optimize()
print(f'Optimal Value: {model.ObjVal}')

x = model.__data
eps = 1.0e-6
optsol = []
for i, j in x:
    if x[i, j].X > eps:
        optsol.append(x[i, j])

print(f'Optimal Solution: {optsol}')

Optimal Value: 6.788226890923063
Optimal Solution: [x_0,1, x_1,3]


## 等式制約に対するラグランジュ緩和問題 (LKNAP)

In [162]:
# CSPを解いた結果を返す関数
def solve_LKNAP(V, E, u, lambda_):
    model = Model('lknap')
    
    ## 変数
    x = {}
    for i, j in E:
        x[i, j] = model.addVar(vtype='B', name=f'x_{i},{j}')
    model.update()
    
    ## 定式化
    ### 制約式
    model.addConstr(quicksum(c[i, j] * x[i, j] for i, j in E) <= u)
 
    ### 目的関数
    model.setObjective(quicksum(l[i, j] * x[i, j] for i, j in E)
                       + lambda_[0] * (1 - quicksum(x[i, j] for i, j in E if i==V[0]))
                       + lambda_[1] * (-1 + quicksum(x[i, j] for i, j in E if j==V[-1]))
                       + quicksum(
                           lambda_[k] * (-quicksum(x[i_, j] for i_, j in [(i_, j) for i_, j in E if i_==i])
                                         + quicksum(x[i_, j] for i_, j in [(j, i_) for j, i_ in E if i_==i])
                                        )
                           for k, i in enumerate(V[1: -1], 2)),
                       GRB.MINIMIZE)
    
    model.update()
    model.__data = x
    
    return model

In [173]:
lambda_ = [0, 0, 0, 0]
count = 1
eps = 1.0e-3
converge = False

while not converge:
    # LKNAPを解く
    model = solve_LKNAP(V, E, u, lambda_=lambda_)
    model.optimize()
    x = model.__data
    
    # 劣勾配
    sub_grad = [
        1 - value(quicksum(x[i, j].X for i, j in E if i==V[0])),
        -1 + value(quicksum(x[i, j].X for i, j in E if j==V[-1]))
    ]
    for i in V[1: -1]:
        sub_grad.append(
            value(
                -quicksum(x[i_, j].X for i_, j in [(i_, j) for i_, j in E if i_==i])
                + quicksum(x[i_, j].X for i_, j in [(j, i_) for j, i_ in E if i_==i])
            )
        )
    
    # ちゃんと収束するようにlambdaの更新幅は徐々に小さくしていく
    # lambdaの更新幅が微小定数epsよりも小さくなれば終了する
    if np.max(np.abs(sub_grad)) / count < eps:
        converge = True
        count += 1
    else:
        lambda_ = [lambda_[i] + sub_grad[i] / count for i in range(len(sub_grad))]
        count += 1

    print(f'{count-1}回目の反複： Objval={model.ObjVal: .4f}')
print(f'lambda={lambda_}')

1回目の反複： Objval= 0.0000
2回目の反複： Objval= 2.0000
3回目の反複： Objval= 3.0000
4回目の反複： Objval= 3.6667
5回目の反複： Objval= 4.0833
6回目の反複： Objval= 4.3667
7回目の反複： Objval= 4.6500
8回目の反複： Objval= 4.8429
9回目の反複： Objval= 5.0607
10回目の反複： Objval= 5.2040
11回目の反複： Objval= 5.3829
12回目の反複： Objval= 5.4750
13回目の反複： Objval= 5.5648
14回目の反複： Objval= 5.6417
15回目の反複： Objval= 5.7186
16回目の反複： Objval= 5.7845
17回目の反複： Objval= 5.8519
18回目の反複： Objval= 5.9095
19回目の反複： Objval= 5.9495
20回目の反複： Objval= 5.9675
21回目の反複： Objval= 5.9553
22回目の反複： Objval= 5.9722
23回目の反複： Objval= 6.0545
24回目の反複： Objval= 6.0197
25回目の反複： Objval= 6.0585
26回目の反複： Objval= 6.0230
27回目の反複： Objval= 6.0999
28回目の反複： Objval= 6.0675
29回目の反複： Objval= 6.1026
30回目の反複： Objval= 6.1413
31回目の反複： Objval= 6.1382
32回目の反複： Objval= 6.1435
33回目の反複： Objval= 6.1402
34回目の反複： Objval= 6.2008
35回目の反複： Objval= 6.1491
36回目の反複： Objval= 6.2025
37回目の反複： Objval= 6.2078
38回目の反複： Objval= 6.2040
39回目の反複： Objval= 6.2356
40回目の反複： Objval= 6.2310
41回目の反複： Objval= 6.2618
42回目の反複： Objval= 6.2566
4

最終的に得られた解

In [174]:
# 最適化
model = solve_LKNAP(V, E, u, lambda_=lambda_)
model.optimize()
print(f'Optimal Value: {model.ObjVal}')

x = model.__data
eps = 1.0e-6
optsol = []
for i, j in x:
    if x[i, j].X > eps:
        optsol.append(x[i, j])

print(f'Optimal Solution: {optsol}')

Optimal Value: 7.000000000000001
Optimal Solution: [x_0,1, x_1,3]
